## <div align="center" style="font-style:italic"> Deteksi Anomali Pupuk Tanah menggunakan Density-Based Spatial Clustering of Applications with Noise </div>
 <h2>Anggota :</h2>
 <ol>
    <li>Farrel Dinarta / 00000055702</li>
    <li>Prudence Tendy / 00000060765</li>
    <li>Christsen Alston Angello / 00000053444</li>
 </ol>

## Sumber Dataset :
1. https://www.kaggle.com/datasets/gdabhishek/fertilizer-prediction
2. https://www.kaggle.com/datasets/srprojects/fertilizer-prediction
3. https://www.kaggle.com/datasets/sanamps/crop-prediction

### Import Libraries

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.decomposition import PCA


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

%matplotlib inline

### Import Datasets

In [169]:
# https://www.kaggle.com/datasets/gdabhishek/fertilizer-prediction
dataset_1 = pd.read_csv("Fertilizer Prediction.csv")
print(dataset_1.info())
print(dataset_1.head())

# print(type(dataset_1.iloc[0]["Temparature"]))
# for _, j in dataset_1.iterrows():

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Temparature      99 non-null     int64 
 1   Humidity         99 non-null     int64 
 2   Moisture         99 non-null     int64 
 3   Soil Type        99 non-null     object
 4   Crop Type        99 non-null     object
 5   Nitrogen         99 non-null     int64 
 6   Potassium        99 non-null     int64 
 7   Phosphorous      99 non-null     int64 
 8   Fertilizer Name  99 non-null     object
dtypes: int64(6), object(3)
memory usage: 7.1+ KB
None
   Temparature  Humidity  Moisture Soil Type  Crop Type  Nitrogen  Potassium  \
0           26        52        38     Sandy      Maize        37          0   
1           29        52        45     Loamy  Sugarcane        12          0   
2           34        65        62     Black     Cotton         7          9   
3           32        62    

In [170]:
# https://www.kaggle.com/datasets/srprojects/fertilizer-prediction
dataset_2 = pd.read_csv("Fertilizer Prediction 2.csv")
print(dataset_2.info())
print(dataset_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  200 non-null    float64
 1   Humidity     200 non-null    float64
 2   Rainfall     200 non-null    float64
 3   pH           200 non-null    float64
 4   N            200 non-null    float64
 5   P            200 non-null    float64
 6   K            200 non-null    float64
 7   Soil         200 non-null    object 
 8   Crop         200 non-null    object 
 9   Fertilizer   200 non-null    object 
dtypes: float64(7), object(3)
memory usage: 15.8+ KB
None
   Temperature  Humidity  Rainfall    pH    N    P    K        Soil     Crop  \
0        24.87     82.84    295.61  6.59  4.0  2.0  2.5      Clayey     rice   
1        28.69     96.65    178.96  6.08  4.0  4.0  4.0    laterite  Coconut   
2        20.27     81.64    270.44  5.01  4.0  4.0  2.0  silty clay     rice   
3        25.0

In [171]:
# https://www.kaggle.com/datasets/sanamps/crop-prediction
dataset_3 = pd.read_csv("Fertilizer Prediction 3.csv")
print(dataset_3.info())
print(dataset_3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB
None
    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


=========================================================================================================
### DATA PREPROCESSING OVERVIEW
 <h3>Feature Selection</h3>
 <ol>
    <li>Nitrogen (ppm)</li>
    <li>Potassium (ppm)</li>
    <li>Phosphorous (ppm)</li>
    <li>Humidity (%)</li>
    <li>Temperature (Celsius)</li>
    <li>Moisture (%)</li>    
    <li>Rainfall (mm)</li>
    <li>pH (0-14)</li>
    <li>Name</li>
 </ol>
 
<h3>Data Cleaning</h3>
<ol>
    <li>Features with incomplete values : </li>
    <table border="1" style="width:70%">
        <tr>
            <th></th>
            <th>Rainfall</th>
            <th>Moisture</th>
            <th>pH</th>
            <th>Name</th>
        </tr>
        <tr>
            <td><b>Dataset 1</b></td>
            <td>Doesn't Exist</td>
            <td>Exists</td>
            <td>Doesn't Exist</td>            
            <td>Exists</td>            
        </tr>
        <tr>
            <td><b>Dataset 2</b></td>
            <td>Exists</td>
            <td>Doesn't Exists</td>
            <td>Exists</td>
            <td>Exists</td>            
        </tr>
        <tr>
            <td><b>Dataset 3</b></td>
            <td>Exists</td>
            <td>Doesn't Exists</td>
            <td>Exists</td>        
            <td>Doesn't Exists</td>            
        </tr>
    </table>
    <li>Since we are merging three datasets and incompleteness on the crucial features seems to be unavoidable, we will predict them using supervised model (Linear Regression)
</ol>
 
<h3>Data Integration</h3>
<ol>
    <li>Integrate the given selected features.</li>
    <li>Remove unneccessary or uncommon features.</li>
    <li>Removed features :</li>
    <ol>
        <li>Crop Type</li>
        <li>Soil Type</li>
    </ol>
</ol>

<h3>Feature Scaling</h3>
 <ol>
    <li>The datasets will be normalized since various units and scale are used.</li>
 </ol>
 
 <h3>Merge Datasets</h3>
 <ol>
    <li>Each preprocessed dataset with synchronized features are merged into one dataset for the DBSCAN modeling.</li>
 </ol>
 
  <h3>Data Segmentation</h3>
 <ol>
    <li>The dataset will be split again according to fertilizer type.</li>
 </ol>

=========================================================================================================

### PREPROCESSING : FEATURE SELECTION

In [172]:
# Synchronize column names
used_features = ['temperature', 'humidity', 'nitrogen', 'potassium', 'phosphorous', 'ph', 'rainfall', 'moisture', 'name']

### Dataset 1

In [173]:
# rename existing columns
dataset_1 = dataset_1.rename(
    columns = {
        'Temparature' : used_features[0], 
        'Humidity' : used_features[1], 
        'Nitrogen' : used_features[2], 
        'Potassium' : used_features[3],
        'Phosphorous' : used_features[4], 
        'Moisture' : used_features[7], 
        'Fertilizer Name' : used_features[8]
    }
)

dataset_1 = dataset_1[used_features[:5] + [used_features[-1], used_features[-2]]]
X_d1 = dataset_1 # for supervised prediction model

# add inexistent columns
new_feature_dataset_1 = pd.DataFrame(columns=used_features[5:7])
dataset_1 = pd.concat([dataset_1, new_feature_dataset_1], axis=1)
dataset_1.head()

,temperature,humidity,nitrogen,potassium,phosphorous,name,moisture,ph,rainfall
0,26,52,37,0,0,Urea,38,NaN,NaN
1,29,52,12,0,36,DAP,45,NaN,NaN
2,34,65,7,9,30,14-35-14,62,NaN,NaN
3,32,62,22,0,20,28-28,34,NaN,NaN
4,28,54,35,0,0,Urea,46,NaN,NaN


### Dataset 2

In [174]:
# rename existing columns
dataset_2 = dataset_2.rename(
    columns = {
        'Temperature' : used_features[0], 
        'Humidity' : used_features[1], 
        'N' : used_features[2], 
        'K' : used_features[3],
        'P' : used_features[4],
        'Rainfall' : used_features[6], 
        'pH' : used_features[5], 
        'Fertilizer' : used_features[8]
    }
)

dataset_2 = dataset_2[used_features[:7] + [used_features[-1]]]
X_d2 = dataset_2 # for supervised prediction model

# add inexistent columns
new_feature_dataset_2 = pd.DataFrame(columns=[used_features[7]])
dataset_2 = pd.concat([dataset_2, new_feature_dataset_2], axis=1)
dataset_2.head()

,temperature,humidity,nitrogen,potassium,phosphorous,ph,rainfall,name,moisture
0,24.87,82.84,4.0,2.5,2.0,6.59,295.61,DAP and MOP,NaN
1,28.69,96.65,4.0,4.0,4.0,6.08,178.96,Good NPK,NaN
2,20.27,81.64,4.0,2.0,4.0,5.01,270.44,MOP,NaN
3,25.07,95.02,2.0,3.5,2.1,5.55,192.90,Urea and DAP,NaN
4,25.04,95.90,2.0,2.1,3.9,6.18,174.80,Urea and MOP,NaN


### Dataset 3

In [175]:
# rename existing columns
dataset_3 = dataset_3.rename(
    columns = {
        'temperature' : used_features[0], 
        'humidity' : used_features[1], 
        'N' : used_features[2], 
        'K' : used_features[3],
        'P' : used_features[4],
        'rainfall' : used_features[6], 
        'ph' : used_features[5]        
    }
)

dataset_3 = dataset_3[used_features[:7]]
X_d3 = dataset_3 # for supervised prediction model

# add inexistent columns
new_feature_dataset_3 = pd.DataFrame(columns=used_features[-2:])
dataset_3 = pd.concat([dataset_3, new_feature_dataset_3], axis=1)
dataset_3.head()

,temperature,humidity,nitrogen,potassium,phosphorous,ph,rainfall,moisture,name
0,20.879744,82.002744,90,43,42,6.502985,202.935536,NaN,NaN
1,21.770462,80.319644,85,41,58,7.038096,226.655537,NaN,NaN
2,23.004459,82.320763,60,44,55,7.840207,263.964248,NaN,NaN
3,26.491096,80.158363,74,40,35,6.980401,242.864034,NaN,NaN
4,20.130175,81.604873,78,42,42,7.628473,262.717340,NaN,NaN


### PREPROCESSING : HANDLE FERTILIZER NAME (CATEGORICAL)

- Get all unique fertilizer name

In [176]:
fertilizer_name_d1 = np.unique(np.array(dataset_1['name']))
fertilizer_name_d2 = np.unique(np.array(dataset_2['name']))
unique_names = np.unique(np.concatenate((fertilizer_name_d1, fertilizer_name_d2)))
unique_names

array(['10-26-26', '14-35-14', '17-17-17', '20-20', '28-28', 'DAP',
       'DAP and MOP', 'Good NPK', 'MOP', 'Urea', 'Urea and DAP',
       'Urea and MOP'], dtype=object)

- Map fertilizer name into integer

In [177]:
name_int_mapping = {name: idx for idx, name in enumerate(unique_names)}

dataset_1['name'] = dataset_1['name'].map(name_int_mapping)
X_d1['name'] = X_d1['name'].map(name_int_mapping)

dataset_2['name'] = dataset_2['name'].map(name_int_mapping)
X_d2['name'] = X_d2['name'].map(name_int_mapping)

name_int_mapping

{'10-26-26': 0,
 '14-35-14': 1,
 '17-17-17': 2,
 '20-20': 3,
 '28-28': 4,
 'DAP': 5,
 'DAP and MOP': 6,
 'Good NPK': 7,
 'MOP': 8,
 'Urea': 9,
 'Urea and DAP': 10,
 'Urea and MOP': 11}

- Inverse Mapping

In [178]:
inv_name_mapping = {v : k for k, v in name_int_mapping.items()}
print(dataset_1['name'].map(inv_name_mapping))
print(dataset_2['name'].map(inv_name_mapping))

0         Urea
1          DAP
2     14-35-14
3        28-28
4         Urea
        ...   
94       28-28
95    10-26-26
96        Urea
97         DAP
98       20-20
Name: name, Length: 99, dtype: object
0       DAP and MOP
1          Good NPK
2               MOP
3      Urea and DAP
4      Urea and MOP
           ...     
195             DAP
196    Urea and MOP
197    Urea and DAP
198             DAP
199    Urea and MOP
Name: name, Length: 200, dtype: object


### PREPROCESSING : FEATURE SCALING (FOR PREDICTION)
- This scaling is intended for the feature predictions from the datasets.
- Normalise all except "name".

In [179]:
scaler = StandardScaler()

name_column1 = dataset_1['name']
name_column2 = dataset_2['name']
name_column3 = dataset_3['name']


td_1 = dataset_1
td_2 = dataset_2
td_3 = dataset_3

original_columns1 = td_1.columns.tolist()
normalized_columns1 = list(filter(lambda x : x not in ["ph", "rainfall"], original_columns1))
td_scaled1 = scaler.fit_transform(td_1[normalized_columns1])
td_1 = pd.DataFrame(td_scaled1, columns=normalized_columns1)
td_1["name"] = name_column1

original_columns2 = td_2.columns.tolist()
normalized_columns2 = list(filter(lambda x : x not in ["moisture"], original_columns2))
td_scaled2 = scaler.fit_transform(td_2[normalized_columns2])
td_2 = pd.DataFrame(td_scaled2, columns=normalized_columns2)
td_2["name"] = name_column2

original_columns3 = td_3.columns.tolist()
normalized_columns3 = list(filter(lambda x : x not in ["moisture", "name"], original_columns3))
td_scaled3 = scaler.fit_transform(td_3[normalized_columns3])
td_3 = pd.DataFrame(td_scaled3, columns=normalized_columns3)
td_3["name"] = name_column3

X_d1 = td_1[used_features[:5] + [used_features[-1], used_features[-2]]]
X_d2 = td_2[used_features[:7] + [used_features[-1]]]
X_d3 = td_3[used_features[:7]]

print("Dataset 1\n", td_1.head(), '\n')
print("Dataset 2\n", td_2.head(), '\n')
print("Dataset 3\n", td_3.head(), '\n')


Dataset 1
    temperature  humidity  nitrogen  potassium  phosphorous  name  moisture
0    -1.229084 -1.230737  1.567539  -0.584910    -1.387607     9 -0.462064
1    -0.368145 -1.230737 -0.598658  -0.584910     1.297209     5  0.162128
2     1.066752  1.006492 -1.031898   0.970777     0.849740     1  1.678023
3     0.492793  0.490209  0.267821  -0.584910     0.103958     4 -0.818745
4    -0.655125 -0.886548  1.394244  -0.584910    -1.387607     9  0.251298 

Dataset 2
    temperature  humidity  nitrogen  potassium  phosphorous        ph  \
0    -0.270737 -0.864575  0.974210  -0.633722    -1.219674  0.610146   
1     1.234233  1.221650  0.974210   1.450890     0.938086 -0.211545   
2    -2.083005 -1.045854  0.974210  -1.328593     0.938086 -1.935486   
3    -0.191943  0.975412 -1.228639   0.756019    -1.111786 -1.065460   
4    -0.203762  1.108350 -1.228639  -1.189619     0.830198 -0.050429   

   rainfall  name  
0  2.042117     6  
1 -0.618406     7  
2  1.468046     8  
3 -0.300466  

### PREPROCESSING : IMPUTING MISSING VALUES

- Using Linear Regression model

### Dataset 1

<ol>
    <li>Predict `rainfall` and `ph` using the third dataset.</li>
<ol>

In [180]:
y_d3 = X_d3[['rainfall', 'ph']]
X_d3 = X_d3.drop(columns=["rainfall", 'ph'], axis=1)

model = LinearRegression()
model.fit(X_d3, y_d3)

LinearRegression()

In [181]:
X_train1 = X_d1.drop(columns=['name', 'moisture']) # drop since still null
y_pred = model.predict(X_train1)
y_pred = np.array(y_pred)

y_pred[:20]

array([[ 1.50935200e-03,  2.44480920e-01],
       [-5.45291140e-02,  5.43693075e-02],
       [-5.59765911e-02, -2.71898372e-01],
       [ 9.72627352e-02,  1.06020687e-01],
       [-2.18275285e-04,  2.10290727e-01],
       [-2.13159841e-01, -2.05310069e-01],
       [-7.17213837e-02,  8.37689889e-02],
       [ 1.30550640e-01,  2.10594461e-01],
       [ 8.80849320e-02,  1.21716567e-01],
       [-6.68122197e-02, -1.52150532e-01],
       [ 3.86743817e-02,  9.84935234e-02],
       [-1.82614162e-01, -3.79658032e-01],
       [-2.41582870e-02,  2.45631287e-01],
       [ 1.05737445e-01,  1.18459433e-01],
       [ 1.96748180e-02,  1.11918831e-01],
       [-1.53246285e-01, -3.43446448e-01],
       [ 1.23258479e-01,  1.92427756e-01],
       [-2.62930271e-02,  1.57910047e-01],
       [ 9.50460555e-03,  2.34513000e-01],
       [ 7.79216797e-02,  8.34264357e-02]])

In [182]:
y_pred2 = model.predict(X_d3)
mse = mean_squared_error(y_d3, y_pred2)
mse

0.971874927011362

- Insert the prediction result into the original dataset

In [183]:
dataset_1["rainfall"] = y_pred[:, 0]
dataset_1["ph"] = y_pred[:, 1]
print(dataset_1.head())

   temperature  humidity  nitrogen  potassium  phosphorous  name  moisture  \
0           26        52        37          0            0     9        38   
1           29        52        12          0           36     5        45   
2           34        65         7          9           30     1        62   
3           32        62        22          0           20     4        34   
4           28        54        35          0            0     9        46   

         ph  rainfall  
0  0.244481  0.001509  
1  0.054369 -0.054529  
2 -0.271898 -0.055977  
3  0.106021  0.097263  
4  0.210291 -0.000218  


### Dataset 2

<ol>
    <li>Predict `moisture` using the first dataset.</li>
<ol>

In [184]:
y_d1 = X_d1[['moisture']]
X_d1 = X_d1.drop(columns=["moisture"], axis=1)

model = LinearRegression()
model.fit(X_d1, y_d1)

LinearRegression()

In [185]:
X_train2 = dataset_2.drop(columns=['moisture','rainfall', 'ph'])
y_pred = model.predict(X_train2)
y_pred = np.array(y_pred)

y_pred[:20]

array([[-0.6472707 ],
       [-0.8743558 ],
       [-1.53127455],
       [ 0.03087253],
       [-0.24962087],
       [-0.90376099],
       [-0.27468798],
       [-0.57658642],
       [-0.24589158],
       [-0.74996871],
       [-0.39154194],
       [-0.95257448],
       [-0.97606553],
       [-0.76721763],
       [-0.35938335],
       [ 0.02623538],
       [-0.74008924],
       [ 0.18617442],
       [ 0.5822412 ],
       [ 0.30140222]])

In [186]:
y_pred2 = model.predict(X_d1)
mse = mean_squared_error(y_d1, y_pred2)
mse

0.9647649413001642

- Insert the prediction result into the original dataset

In [187]:
dataset_2["moisture"] = y_pred[:, 0]
print(dataset_2.head())

   temperature  humidity  nitrogen  potassium  phosphorous    ph  rainfall  \
0        24.87     82.84       4.0        2.5          2.0  6.59    295.61   
1        28.69     96.65       4.0        4.0          4.0  6.08    178.96   
2        20.27     81.64       4.0        2.0          4.0  5.01    270.44   
3        25.07     95.02       2.0        3.5          2.1  5.55    192.90   
4        25.04     95.90       2.0        2.1          3.9  6.18    174.80   

   name  moisture  
0     6 -0.647271  
1     7 -0.874356  
2     8 -1.531275  
3    10  0.030873  
4    11 -0.249621  


### Dataset 3

<ol>
    <li>Predict `name` using the first and second dataset.</li>
<ol>

In [188]:
# merge dataset 1 and 2
dataset_1 = dataset_1.astype(float)
dataset_2 = dataset_2.astype(float)
dataset_3 = dataset_3.astype(float)
merged_dataset = pd.merge(dataset_1, dataset_2, how='outer')

y = merged_dataset[['name']]
X = merged_dataset.drop(columns=["name", 'moisture'], axis=1)

model = LogisticRegression(max_iter=10000, multi_class='ovr', random_state=42)
model.fit(X, y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=10000, multi_class='ovr', random_state=42)

In [189]:
X_train3 = dataset_3.drop(columns=['name','moisture'])
y_pred = model.predict(X_train3)
y_pred = np.array(y_pred)
np.unique(y_pred[:])

array([0., 5., 7.])

In [190]:
y_pred2 = model.predict(X)
accuracy = accuracy_score(y, y_pred2)
accuracy

0.8127090301003345

- Insert the prediction result into the original dataset

In [191]:
# dataset_3["moisture"] = y_pred[:, 0]
dataset_3["name"] = y_pred[:].round().astype(int)
print(dataset_3.head())

   temperature   humidity  nitrogen  potassium  phosphorous        ph  \
0    20.879744  82.002744      90.0       43.0         42.0  6.502985   
1    21.770462  80.319644      85.0       41.0         58.0  7.038096   
2    23.004459  82.320763      60.0       44.0         55.0  7.840207   
3    26.491096  80.158363      74.0       40.0         35.0  6.980401   
4    20.130175  81.604873      78.0       42.0         42.0  7.628473   

     rainfall  moisture  name  
0  202.935536       NaN     7  
1  226.655537       NaN     7  
2  263.964248       NaN     7  
3  242.864034       NaN     7  
4  262.717340       NaN     7  


- Predict `moisture` using the first and second dataset

In [192]:
name_col = merged_dataset["name"]
norm_columns = list(filter(lambda x : x != "name", original_columns1))
t_merged_dataset = scaler.fit_transform(merged_dataset[norm_columns])
t_merged_dataset = pd.DataFrame(t_merged_dataset, columns=norm_columns)
t_merged_dataset["name"] = name_col

y = t_merged_dataset[['moisture']]
X = t_merged_dataset.drop(columns=["name", 'moisture'], axis=1)

model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [193]:
print(td_3)
X_train3 = td3.drop(columns=['name', 'moisture'])

y_pred = model.predict(X_train3)
y_pred = np.array(y_pred)
y_pred[:20]

array([[-15.16232479],
       [-14.19346773],
       [-17.80849344],
       [-17.41795968],
       [-15.50922129],
       [-14.55300296],
       [-13.90803163],
       [-18.3957283 ],
       [-16.98379683],
       [-16.1690975 ],
       [-15.21569089],
       [-17.33363228],
       [-14.5903503 ],
       [-17.47859149],
       [-14.70066684],
       [-17.23272017],
       [-18.3158247 ],
       [-14.26077839],
       [-15.61575943],
       [-16.29164532]])

In [194]:
y_pred2 = model.predict(X)
mse = mean_squared_error(y, y_pred2)
mse

0.09973377854818619

- Insert the prediction result into the original dataset

In [195]:
dataset_3["moisture"] = y_pred[:]
dataset_3.head()

ValueError: Length of values (299) does not match length of index (2200)

### PREPROCESSING : MERGE ALL DATASET
- Merge and filter irrelevant values

In [ ]:
# merge
merged_dataset["name"] = merged_dataset["name"].round().astype(int)
merged_dataset = pd.merge(merged_dataset, dataset_3, how='outer')

# filter 
merged_dataset = merged_dataset[(merged_dataset["name"] >= 0) | (merged_dataset["moisture"] >= 0)]
print(merged_dataset.shape)

### PREPROCESSING : FEATURE SCALING ON MERGED DATASET
- Normalise all except "name"

In [ ]:
scaler = MinMaxScaler()

unormalized_dataset = merged_dataset
name_column = merged_dataset['name']


original_columns = merged_dataset.columns.tolist()
normalized_columns = list(filter(lambda x : x != "name", original_columns))

merged_dataset_scaled = scaler.fit_transform(merged_dataset[normalized_columns])
merged_dataset = pd.DataFrame(merged_dataset_scaled, columns=normalized_columns)

merged_dataset["name"] = name_column

merged_dataset = merged_dataset.dropna(subset=['name'])
      
merged_dataset["name"] = merged_dataset["name"].round().astype(int)

print(merged_dataset.info())
# print(merged_dataset.isna().sum())
# print(np.isinf(merged_dataset).sum())

### PREPROCESSING : DATA SEGMENTATION

In [ ]:
for key, value in name_int_mapping.items():
    count = len(merged_dataset[merged_dataset["name"] == value])
    print("Found", count, key + "'s Fertilizers with type ID : " + str(value))

- Separate dataset according to its own type

In [ ]:
def dataset_filter(type): # type : Integer
    return merged_dataset[merged_dataset["name"] == type]

# access each fertilizer type through its original name not mapped ID
fertilizer_dataset = {key : dataset_filter(value) for key, value in name_int_mapping.items()}

### MODELING : DBSCAN
- Higher eps gives better silhouette score, but might impact the clustering result since it will be too broad.

In [ ]:
from sklearn.cluster import DBSCAN

silhouette_avgs = []

def inverse_normalize(normalized_value, min_val, max_val):
    denormalized_value = (normalized_value * (max_val - min_val)) + min_val
    return denormalized_value

def dbscan_analysis(data, key):
    db = DBSCAN(eps=0.5, min_samples=data.shape[1]+1).fit(data)
    labels = db.fit_predict(data)
    
    min_max_list = {}
    
    for feature in unormalized_dataset.columns:
        values = unormalized_dataset[feature].values 
        max_val = np.max(values, axis=0)
        min_val = np.min(values, axis=0)
        min_max_list[feature] = {"max" : max_val, "min" : min_val}
        

    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(data)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, s=8, cmap='viridis')
    plt.title('DBSCAN (2D PCA Projection) : %s' % key)
    plt.show()    
    
    cluster_details = pd.DataFrame({'Label': labels, 'DataPoint': range(len(labels))})
    cluster_counts = cluster_details.groupby('Label').count()
    
    cluster_data_points = {label: data[labels == label] for label in np.unique(labels)}
    
    for label, points in cluster_data_points.items():
        print(f"Label {label}: {len(points)} data points")
        avg_values = np.mean(points, axis=0)
        print(f"Averages for Label {label} (Normalized Value):\n", avg_values, "\n")
        print(f"Averages for Label {label} (Original Value):")
        for feature, avg_value in enumerate(avg_values):
            print(data.columns[feature], str(inverse_normalize(avg_value, min_max_list[data.columns[feature]]["min"], min_max_list[data.columns[feature]]["max"])))        
        print("============================")
        
    
    unique_labels = np.unique(labels)
    if len(unique_labels) > 1:
        silhouette_avg = silhouette_score(data, labels)
        silhouette_avgs.append(silhouette_avg)
        print("Silhouette Score:", silhouette_avg)
    else:
        print("Only one cluster. Silhouette score cannot be computed.")

    # positive integer = cluster member, negative = outlier / noise

In [ ]:
for key, value in fertilizer_dataset.items():
    dbscan_analysis(data=value, key=key)    

### MODELING : PERFORMANCE TEST

- Using Silhouette Score 
<ul>
    <li>1 means good separation between clusters</li>
    <li>0 means possibility of overlapping cluster</li>
    <li>-1 means poor cluster separation</li>
</ul>

In [ ]:
print("Silhouette Score Average : ", np.mean(silhouette_avgs))

### CONCLUSION
1. Hasil prediksi pada feature pada tahap preprocessing menghasilkan data-data yang dapat dibagi sesuai tiap pupuk, dan hasil model pelatihannya bersifat dominan pada beberapa jenis pupuk saja.
2. Jarang ditemukan noise yang ditemukan pada hasil clustering pada masing-masing tipe pupuk, namun pada dataset yang jumlahnya cukup besar terkadang muncul dua cluster non-outlier. Apabila dianalisis rata-rata value feature pada cluster tersebut, maka terdapat perbedaan komposisi yang cukup drastis. Hal ini dapat dijadikan referensi sebagai penanda adanya anomali pada satu cluster atau cluster yang lainnya, karena seharusnya masing-masing tipe pupuk memiliki konsistensi untuk komposisi dan karakteristiknya.